In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as mplt
import matplotlib as plt
import datetime as dati

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV

In [7]:
dataset = pd.read_csv("/home/federico/Git/datos-tp2-2c2020/data/friofrio/Train_TP2_Datos_2020-2C.csv",
        parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                       'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
dataset = dataset.drop(columns= ['Prod_Category_A',
                                 'Product_Category_B',
                                 'Actual_Delivery_Date',
                                 'Submitted_for_Approval',
                                 'Last_Activity','Price',
                                 'Currency','Brand',
                                 'Size','Product_Type'])

In [9]:
dataset = dataset.loc[(dataset['Stage'] == 'Closed Won') | (dataset['Stage'] == 'Closed Lost')]

In [14]:
dataset['count'] = 1
dataset['TRF promedio'] = dataset['TRF']
dataset['TRF estandar'] = dataset['TRF']
oportunidades = dataset.groupby('Opportunity_ID').agg({
    'Total_Amount': sum , 'Stage': 'first', 'count': 'count',
    'Opportunity_Owner':'first','Opportunity_Created_Date': 'first','TRF promedio':'mean',
    'TRF estandar':'std'}).reset_index()

oportunidades['Grupo'] = oportunidades.Stage.shift().ne(dataset['Stage']).cumsum()
oportunidades_owner = oportunidades.sort_values(by=['Opportunity_Owner', 'Opportunity_Created_Date'])
oportunidades_owner['Grupo'] = oportunidades_owner.groupby('Opportunity_Owner').Stage.shift().ne(dataset['Stage']).cumsum()
oportunidades_owner.groupby(['Opportunity_Owner', 'Grupo']).size().reset_index(level=1, drop=True)

oportunidades_won_owner = oportunidades_owner.loc[oportunidades_owner['Stage']== 'Closed Won']
oportunidades_loss_owner = oportunidades_owner.loc[oportunidades_owner['Stage']== 'Closed Lost']

oportunidades_won =oportunidades_owner.loc[(oportunidades_owner['Stage']=='Closed Won')]
oportunidades_lost =oportunidades_owner.loc[(oportunidades_owner['Stage']=='Closed Lost')]

grouped_won_owner= oportunidades_won_owner.groupby(['Grupo','Opportunity_Owner']).agg({'count':'count'}).reset_index()
grouped_loss_owner= oportunidades_loss_owner.groupby(['Grupo','Opportunity_Owner']).agg({'count':'count'}).reset_index()

Oportunity_owners_top = grouped_won_owner.nlargest(10,'count')
Oportunity_owners_worst = grouped_loss_owner.nlargest(10,'count')

In [87]:
prueba = oportunidades.merge(grouped_won_owner.reset_index(), how = 'inner', on = ['Opportunity_Owner','Grupo'])
#prueba = oportunidades.merge(grouped_loss_owner.reset_index(), how = 'inner', on = ['Opportunity_Owner','Grupo'])
prueba.head()

,Opportunity_ID,count_x,TRF estandar,Total_Amount,TRF promedio,Opportunity_Created_Date,Opportunity_Owner,Stage,Grupo,index,count_y
0,8,5,0.0,470400.0,0.0,2015-12-09,Person_Name_50,Closed Lost,5,5,6
1,9,5,0.0,559320.0,0.0,2015-12-09,Person_Name_50,Closed Lost,5,5,6
2,10,5,0.0,579600.0,0.0,2015-12-09,Person_Name_50,Closed Lost,5,5,6
3,11,1,NaN,1621851.0,3.0,2015-12-09,Person_Name_49,Closed Won,5,4,1
4,25,1,NaN,1556587.5,3.0,2015-12-10,Person_Name_18,Closed Lost,10,6,1


In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV

In [88]:
prueba = prueba.select_dtypes(include = np.number).fillna(0).drop(columns = 'count_x')
prueba.head()

,Opportunity_ID,TRF estandar,Total_Amount,TRF promedio,Grupo,index,count_y
0,8,0.0,470400.0,0.0,5,5,6
1,9,0.0,559320.0,0.0,5,5,6
2,10,0.0,579600.0,0.0,5,5,6
3,11,0.0,1621851.0,3.0,5,4,1
4,25,0.0,1556587.5,3.0,10,6,1


In [89]:
dataset['Oportunidad ganada'] = (dataset['Stage'] == 'Closed Won').astype(int)

In [90]:
prueba_train = dataset.drop(columns = ['count', 'TRF estandar', 'TRF promedio']).merge(prueba[['count_y', 'Opportunity_ID']], on = 'Opportunity_ID', how = 'left').select_dtypes(include = np.number).fillna(0)
prueba_train.nlargest(5, 'count_y')

,ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,ASP,ASP_(converted),Delivery_Year,TRF,Total_Amount,Total_Taxable_Amount,Oportunidad ganada,count_y
88,24326,1,0,0,0,37,0.0,0.00000,2016,0,23400.0,599880.0,0,8.0
89,24327,1,0,0,0,37,0.0,0.00000,2016,0,4400.0,599880.0,0,8.0
90,24328,1,0,0,0,37,0.0,0.00000,2016,0,6200.0,599880.0,0,8.0
91,24329,1,0,0,0,37,0.0,0.00000,2016,0,140000.0,599880.0,0,8.0
92,24330,1,0,0,0,37,78.0,0.70125,2016,0,425880.0,599880.0,0,8.0


In [91]:
X = prueba_train.drop(columns = 'Oportunidad ganada').fillna(0)

y = prueba_train[['Oportunidad ganada']].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [92]:
X_train.head()

,ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,ASP,ASP_(converted),Delivery_Year,TRF,Total_Amount,Total_Taxable_Amount,count_y
9798,22022,1,1,0,0,6206,0.390,0.390,2017,0,79170.0,79170.0,0.0
10709,26482,1,1,0,0,6637,0.000,0.000,2017,0,6200.0,620175.0,0.0
5593,27979,1,0,1,0,3418,0.365,0.365,2017,21,7703325.0,0.0,0.0
16304,26159,1,0,1,0,12531,0.000,0.000,2016,0,12800.0,6995400.0,0.0
4725,24447,1,1,0,0,2787,0.000,0.000,2016,0,6200.0,628000.0,0.0


In [102]:
param_grid = {
    'max_depth': [20, 25],
    'max_features': [3, 4],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2],
    'n_estimators': [100]
}
rf_model = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf_model, param_grid = param_grid)
grid_search.fit(X_train, y_train)
grid_search.best_params_

/home/federico/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/federico/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/federico/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/federico/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: Dat

{'max_depth': 25,
 'max_features': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [103]:
rf_model = RandomForestRegressor(n_estimators = 100,
                                 max_features = 3,
                                 max_depth = 25,
                                 min_samples_leaf = 1,
                                 min_samples_split = 2)
rf_model.fit(X_train, y_train)

preds = (rf_model.predict(X_test))

log_loss(y_test, preds, eps=1e-15, normalize=True, sample_weight=None, labels=None)

/home/federico/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.28433682940255306

In [104]:
test_csv = pd.read_csv("/home/federico/Git/datos-tp2-2c2020/data/friofrio/Test_TP2_Datos_2020-2C.csv",
        parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                       'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
test_csv = test_csv.drop(columns= ['Prod_Category_A',
                                 'Product_Category_B',
                                 'Actual_Delivery_Date',
                                 'Submitted_for_Approval',
                                 'Last_Activity','Price',
                                 'Currency','Brand',
                                 'Size','Product_Type'])

In [105]:
X_test_to_submit = test_csv.select_dtypes(include = np.number).fillna(0)
X_test_to_submit = X_test_to_submit.merge(
            prueba[['count_y', 'Opportunity_ID']], 
            on = 'Opportunity_ID', how = 'left').select_dtypes(include = np.number).fillna(0)

In [106]:
preds_to_submit = rf_model.predict(X_test_to_submit)

test_csv['Target'] = preds_to_submit

result = test_csv[['Opportunity_ID','Target']].groupby(['Opportunity_ID']).mean()
result.reset_index(inplace=True)

result['Opportunity_ID'] = result['Opportunity_ID'].astype(int)
result.dtypes

result.to_csv('predicciones.csv', index = False)

# Verifico features de aprobaciones

In [78]:
dataset_approval = pd.read_csv("/home/federico/Git/datos-tp2-2c2020/data/friofrio/Train_TP2_Datos_2020-2C.csv",
        parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                       'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
dataset_approval = dataset_approval.drop(columns= ['Prod_Category_A',
                                 'Product_Category_B',
                                 'Actual_Delivery_Date',
                                 'Submitted_for_Approval',
                                 'Last_Activity','Price',
                                 'Currency','Brand',
                                 'Size','Product_Type'])

In [80]:
dataset_approval = dataset_approval.loc[(dataset_approval['Stage'] == 'Closed Won') | (dataset_approval['Stage'] == 'Closed Lost')]

dataset_approval['Precio necesita aprobacion'] = (dataset_approval['Pricing, Delivery_Terms_Quote_Appr'] == 1)
dataset_approval['Precio aprobado'] = (dataset_approval['Pricing, Delivery_Terms_Approved'] == 1)
dataset_approval['Precio necesita aprobacion'] = dataset_approval['Precio necesita aprobacion'].astype(int)
dataset_approval['Precio aprobado'] = dataset_approval['Precio aprobado'].astype(int)
dataset_approval = dataset_approval.drop(columns = ['Pricing, Delivery_Terms_Quote_Appr', 'Pricing, Delivery_Terms_Approved'])



a = dataset_approval.groupby('Opportunity_ID').agg({'Precio necesita aprobacion':'sum',
                                                'Precio aprobado':'sum'})
a.reset_index(inplace=True)

dataset_approval = dataset_approval.drop(
                columns = ['Precio aprobado','Precio necesita aprobacion']).merge(a, on = 'Opportunity_ID')
dataset_approval.head()

dataset_approval['Stage'] = (dataset_approval['Stage'] == 'Closed Won').astype(int)

dataset_approval = dataset_approval.select_dtypes(include = np.number).fillna(0)

In [86]:
X = dataset_approval.drop(columns = ['Stage', 'Opportunity_ID', 'ID']).fillna(0)

y = dataset_approval[['Stage']].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [87]:
X_train.head()

,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,ASP,ASP_(converted),Delivery_Year,TRF,Total_Amount,Total_Taxable_Amount,Precio necesita aprobacion,Precio aprobado
9798,0,0,0.390,0.390,2017,0,79170.0,79170.0,1,1
10709,0,0,0.000,0.000,2017,0,6200.0,620175.0,6,6
5593,1,0,0.365,0.365,2017,21,7703325.0,0.0,1,0
16304,1,0,0.000,0.000,2016,0,12800.0,6995400.0,6,0
4725,0,0,0.000,0.000,2016,0,6200.0,628000.0,5,5


In [88]:
rf_model_1 = RandomForestRegressor(n_estimators = 300,
                                 #max_features = 80,
                                 #max_depth = 35,
                                 min_samples_leaf = 1,
                                 min_samples_split = 3)
rf_model_1.fit(X_train, y_train)

preds = (rf_model_1.predict(X_test))

log_loss(y_test, preds, eps=1e-15, normalize=True, sample_weight=None, labels=None)

/home/federico/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.3374854597100236

In [89]:
feature_importances_1 = \
    pd.DataFrame(rf_model_1.feature_importances_, \
                 index = X_train.columns, \
                 columns = ['importance'])
feature_importances_1

,importance
Bureaucratic_Code_0_Approval,0.020036
Bureaucratic_Code_0_Approved,0.008034
ASP,0.133772
ASP_(converted),0.122873
Delivery_Year,0.042482
TRF,0.189868
Total_Amount,0.163977
Total_Taxable_Amount,0.171916
Precio necesita aprobacion,0.073975
Precio aprobado,0.073068


In [96]:
test_csv = pd.read_csv("/home/federico/Git/datos-tp2-2c2020/data/friofrio/Test_TP2_Datos_2020-2C.csv",
        parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                       'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
test_csv = test_csv.drop(columns= ['Prod_Category_A',
                                 'Product_Category_B',
                                 'Actual_Delivery_Date',
                                 'Submitted_for_Approval',
                                 'Last_Activity','Price',
                                 'Currency','Brand',
                                 'Size','Product_Type'])

In [101]:
X_test_to_submit = test_csv
X_test_to_submit['Precio necesita aprobacion'] = (X_test_to_submit['Pricing, Delivery_Terms_Quote_Appr'] == 1)
X_test_to_submit['Precio aprobado'] = (X_test_to_submit['Pricing, Delivery_Terms_Approved'] == 1)
X_test_to_submit['Precio necesita aprobacion'] = X_test_to_submit['Precio necesita aprobacion'].astype(int)
X_test_to_submit['Precio aprobado'] = X_test_to_submit['Precio aprobado'].astype(int)
X_test_to_submit = X_test_to_submit.drop(columns = ['Pricing, Delivery_Terms_Quote_Appr',
                                                    'Pricing, Delivery_Terms_Approved', 'ID'])

a = X_test_to_submit.groupby('Opportunity_ID').agg({'Precio necesita aprobacion':'sum',
                                                'Precio aprobado':'sum'})
a.reset_index(inplace=True)

X_test_to_submit = X_test_to_submit.drop(
                columns = ['Precio aprobado','Precio necesita aprobacion']).merge(a, on = 'Opportunity_ID')

X_test_to_submit = X_test_to_submit.select_dtypes(include = np.number).fillna(0)

X_test_to_submit = X_test_to_submit.drop(columns = ['Opportunity_ID'])

X_test_to_submit.head()

,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,ASP,ASP_(converted),Delivery_Year,TRF,Total_Amount,Total_Taxable_Amount,Precio necesita aprobacion,Precio aprobado
0,1,1,0.325,0.36761,2019,0,162240.0,367419.0,3,3
1,1,1,0.320,0.36195,2019,0,78624.0,367419.0,3,3
2,1,1,0.275,0.31105,2019,0,126555.0,367419.0,3,3
3,1,1,0.325,0.36761,2019,1,243360.0,757783.5,5,5
4,1,1,0.320,0.36195,2019,0,157248.0,757783.5,5,5


In [102]:
preds_to_submit = rf_model_1.predict(X_test_to_submit)

test_csv['Target'] = preds_to_submit.astype('float32')

result = test_csv[['Opportunity_ID','Target']].groupby(['Opportunity_ID']).mean()
result.reset_index(inplace=True)

result['Opportunity_ID'] = result['Opportunity_ID'].astype(int)

result.to_csv('predicciones_pruebas.csv', index = False)

# Verifico features de Quarters

In [103]:
dataset_quarter = pd.read_csv("/home/federico/Git/datos-tp2-2c2020/data/friofrio/Train_TP2_Datos_2020-2C.csv",
        parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                       'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
dataset_quarter = dataset_quarter.drop(columns= ['Prod_Category_A',
                                 'Product_Category_B',
                                 'Actual_Delivery_Date',
                                 'Submitted_for_Approval',
                                 'Last_Activity','Price',
                                 'Currency','Brand',
                                 'Size','Product_Type'])

In [104]:
dataset_quarter.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage
0,27761,EMEA,None,1,1,1,1,Bureaucratic_Code_4,2015-06-16,None,...,2016-06-30,2016 - 5,Q2,2016,10,EUR,5272800.0,EUR,5272800.0,Closed Lost
1,27760,EMEA,None,0,0,0,0,Bureaucratic_Code_4,2015-06-16,None,...,2016-01-20,2016 - 1,Q1,2016,0,EUR,48230.0,EUR,48230.0,Closed Won
2,27446,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,2016-01-25,2016 - 1,Q1,2016,0,USD,83865.6,USD,83865.6,Closed Won
3,16808,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,2018-03-31,2018 - 2,Q1,2018,14,USD,7421881.5,USD,7421881.5,Closed Lost
4,16805,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,2018-02-28,2018 - 2,Q1,2018,25,USD,13357192.5,USD,13357192.5,Closed Lost


In [144]:
pivoted = dataset_quarter[['Opportunity_ID','Delivery_Quarter']].pivot_table(
                    index = 'Opportunity_ID',
                    columns = 'Delivery_Quarter',
                    aggfunc = {'Delivery_Quarter':'count'})\
                    .fillna(0).reset_index()

pivoted = pivoted.set_index('Opportunity_ID')
pivoted.columns = pivoted.columns.droplevel(0)
pivoted = pivoted.reset_index()
pivoted.head()

approval_quarter = dataset_approval.merge(pivoted, on = 'Opportunity_ID')

In [173]:
X = approval_quarter.drop(columns = ['Stage', 'Opportunity_ID', 'ID']).fillna(0)

y = approval_quarter[['Stage']].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [175]:
rf_model_1 = RandomForestRegressor(n_estimators = 300,
                                 #max_features = 80,
                                 #max_depth = 35,
                                 min_samples_leaf = 1,
                                 min_samples_split = 3)
rf_model_1.fit(X_train, y_train)

preds = (rf_model_1.predict(X_test))

log_loss(y_test, preds, eps=1e-15, normalize=True, sample_weight=None, labels=None)

/home/federico/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.3197409404907338

In [178]:
feature_importances_1 = \
    pd.DataFrame(rf_model_1.feature_importances_, \
                 index = X_train.columns, \
                 columns = ['importance'])
feature_importances_1.sort_values(by = 'importance')

,importance
Bureaucratic_Code_0_Approved,0.007859
Bureaucratic_Code_0_Approval,0.016583
Q1,0.022556
Q3,0.023366
Q2,0.024436
Q4,0.028616
Delivery_Year,0.038789
Precio necesita aprobacion,0.064492
Precio aprobado,0.069197
ASP_(converted),0.106763


In [179]:
test_csv = pd.read_csv("/home/federico/Git/datos-tp2-2c2020/data/friofrio/Test_TP2_Datos_2020-2C.csv",
        parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                       'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
test_csv = test_csv.drop(columns= ['Prod_Category_A',
                                 'Product_Category_B',
                                 'Actual_Delivery_Date',
                                 'Submitted_for_Approval',
                                 'Last_Activity','Price',
                                 'Currency','Brand',
                                 'Size','Product_Type'])

In [180]:
X_test_to_submit = test_csv
X_test_to_submit['Precio necesita aprobacion'] = (X_test_to_submit['Pricing, Delivery_Terms_Quote_Appr'] == 1)
X_test_to_submit['Precio aprobado'] = (X_test_to_submit['Pricing, Delivery_Terms_Approved'] == 1)
X_test_to_submit['Precio necesita aprobacion'] = X_test_to_submit['Precio necesita aprobacion'].astype(int)
X_test_to_submit['Precio aprobado'] = X_test_to_submit['Precio aprobado'].astype(int)
X_test_to_submit = X_test_to_submit.drop(columns = ['Pricing, Delivery_Terms_Quote_Appr',
                                                    'Pricing, Delivery_Terms_Approved', 'ID'])

a = X_test_to_submit.groupby('Opportunity_ID').agg({'Precio necesita aprobacion':'sum',
                                                'Precio aprobado':'sum'})
a.reset_index(inplace=True)

X_test_to_submit = X_test_to_submit.drop(
                columns = ['Precio aprobado','Precio necesita aprobacion']).merge(a, on = 'Opportunity_ID')

X_test_to_submit = X_test_to_submit.select_dtypes(include = np.number).fillna(0)



pivoted_test = test_csv[['Opportunity_ID','Delivery_Quarter']].pivot_table(
                    index = 'Opportunity_ID',
                    columns = 'Delivery_Quarter',
                    aggfunc = {'Delivery_Quarter':'count'})\
                    .fillna(0).reset_index()
pivoted_test = pivoted_test.set_index('Opportunity_ID')
pivoted_test.columns = pivoted_test.columns.droplevel(0)
pivoted_test = pivoted_test.reset_index()
X_test_to_submit = X_test_to_submit.merge(pivoted_test, on = 'Opportunity_ID')


X_test_to_submit = X_test_to_submit.drop(columns = ['Opportunity_ID'])
X_test_to_submit.head()

,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,ASP,ASP_(converted),Delivery_Year,TRF,Total_Amount,Total_Taxable_Amount,Precio necesita aprobacion,Precio aprobado,Q1,Q2,Q3,Q4
0,1,1,0.325,0.36761,2019,0,162240.0,367419.0,3,3,0.0,3.0,0.0,0.0
1,1,1,0.320,0.36195,2019,0,78624.0,367419.0,3,3,0.0,3.0,0.0,0.0
2,1,1,0.275,0.31105,2019,0,126555.0,367419.0,3,3,0.0,3.0,0.0,0.0
3,1,1,0.325,0.36761,2019,1,243360.0,757783.5,5,5,0.0,5.0,0.0,0.0
4,1,1,0.320,0.36195,2019,0,157248.0,757783.5,5,5,0.0,5.0,0.0,0.0


In [181]:
preds_to_submit = rf_model_1.predict(X_test_to_submit)

test_csv['Target'] = preds_to_submit.astype('float32')

result = test_csv[['Opportunity_ID','Target']].groupby(['Opportunity_ID']).mean()
result.reset_index(inplace=True)

result['Opportunity_ID'] = result['Opportunity_ID'].astype(int)

result.to_csv('predicciones_pruebas.csv', index = False)